In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import imutils
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead.")

Using GPU: NVIDIA GeForce RTX 2070 SUPER


In [3]:
labels = ['glioma', 'meningioma', 'notumor', 'pituitary']

x_train = [] # training images.
y_train  = [] # training labels.
x_test = [] # testing images.
y_test = [] # testing labels.

image_size = 150

for label in labels:
    trainPath = os.path.join('raw_data/Training',label)
    for file in tqdm(os.listdir(trainPath)):
        image = cv2.imread(os.path.join(trainPath, file), 0) # load images in gray.
        image = cv2.bilateralFilter(image, 2, 50, 50) # remove images noise.
        #image = cv2.applyColorMap(image, cv2.COLORMAP_BONE) # produce a pseudocolored image.
        image = cv2.resize(image, (image_size, image_size)) # resize images into 200*200.
        x_train.append(image)
        y_train.append(1 if label != 'notumor' else 0)
    
    testPath = os.path.join('raw_data/Testing',label)
    for file in tqdm(os.listdir(testPath)):
        image = cv2.imread(os.path.join(testPath, file), 0)
        image = cv2.bilateralFilter(image, 2, 50, 50)
        #image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
        image = cv2.resize(image, (image_size, image_size))
        x_test.append(image)
        y_test.append(1 if label != 'notumor' else 0)


x_train = np.array(x_train) / 255.0 # normalize Images into range 0 to 1.
x_test = np.array(x_test) / 255.0

print(x_train.shape)
print(x_test.shape)

100%|██████████| 300/300 [00:00<00:00, 308.51it/s]


(5712, 150, 150)
(1311, 150, 150)


In [4]:
# Convert numpy arrays to PyTorch tensors
train_images_tensor = torch.tensor(x_train).float() / 255.0  # Normalize if not already done
train_labels_tensor = torch.tensor(np.array(y_train)).long()  # Ensure labels are in long format for CE Loss

test_images_tensor = torch.tensor(x_test).float() / 255.0
test_labels_tensor = torch.tensor(np.array(y_test)).long()

# Reshape the tensors to add a channel dimension (1, 150, 150)
train_images_tensor = train_images_tensor.unsqueeze(1)
test_images_tensor = test_images_tensor.unsqueeze(1)

# Create TensorDatasets
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True, num_workers=3)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True, num_workers=3)